<a href="https://colab.research.google.com/github/PrajwalSathyanarayana/TensorFlow/blob/main/Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch #import PyTorch library
torch.Tensor #refers to Tensor class within the torch library.
# A Tensor is a fundamental data structure in PyTorch; it's a multi dimensional array,
# similar to NumPy arrays.

torch.Tensor

In [3]:
# Create a new tensor in PyTorch.
x = torch.tensor([1.0, 2.0], requires_grad=True)
print(x)

notes = '''
[1.0, 2.0] - data that tensor x will hold.
requires_grad = True - important argument that enables automatic differentiation.
It allows PyTorch to track all operations performed on a tensor, and automatically computes the
gradient of any function with respect to this tensor later on.
'''

tensor([1., 2.], requires_grad=True)


In [4]:
y = x * 2
z = y.sum()
print(y)
print(z)
notes = '''
- y = x*2 - performs element wise multiplication of tensor x by scalar value 2.
- [1.0, 2.0] * 2 = [2.0, 4.0]
- Python automatically tracks this multiplication operation;
indicating that this tensor y was created as a result of a multiplication operation and PyTorch
can compute gradients with respect to it.
- z = y.sum() - computes the sum of all elements in the tensor y.
- 2.0 + 4.0 = 6.0 i.e., single element tensor.
- This sum operation is also tracked.
'''

tensor([2., 4.], grad_fn=<MulBackward0>)
tensor(6., grad_fn=<SumBackward0>)


In [5]:
print(z.backward())

None


In [6]:
torch.__version__

'2.9.0+cu128'

In [7]:
torch.cuda.is_available

<function torch.cuda.is_available() -> bool>

In [8]:
tensor0d = torch.tensor(1) # scalar; 0D tensor
tensor1d = torch.tensor([1,2,3]) # vector; 1D tensor
tensor2d = torch.tensor([[1,2],
                         [3,4]]) # matrix; 2D tensor - rows and columns
tensor3d = torch.tensor([[[1,2], [3,4]],
                         [[5,6], [7,8]]]) # tensor; 3D tensor - collection of 2D matrics
print(tensor0d)
print(tensor1d)
print(tensor2d)
print(tensor3d)

tensor(1)
tensor([1, 2, 3])
tensor([[1, 2],
        [3, 4]])
tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])


In [9]:
print(tensor0d.dtype)

torch.int64


In [10]:
#LOGISTIC REGRESSION FORWARD PASS
import torch.nn.functional as F #import the functional module containing various activation, loss functions and other operations
y = torch.tensor([1.0]) # true label i.e., actual output
x1 = torch.tensor([1.1]) # input feature
w1 = torch.tensor([2.2], requires_grad = True) # weight parameter;
# requires_grad = true tracks all operations performed on 'w1' tensor.
b = torch.tensor([0.0],  requires_grad = True) # bias unit
# requires_grad = true tracks all operations performed on 'b' tensor.
z = x1 * w1 + b # net input i.e., 1.1 * 2.2 + 0.0 = 2.42
a = torch.sigmoid(z) # applies sigmoid activation function to net input 'z'; sigmoid squashes any real valued number into a
# range between 0 and 1, making it suitable for binary classification probabilites. 'a' represents the predicted output or
# activation of logistic regression model.
loss = F.binary_cross_entropy(a,y) # calculate the binary cross entropy loss between predicted output 'a' and true label 'y'
print(z)
print(a)
print(loss)

tensor([2.4200], grad_fn=<AddBackward0>)
tensor([0.9183], grad_fn=<SigmoidBackward0>)
tensor(0.0852, grad_fn=<BinaryCrossEntropyBackward0>)


In [11]:
from torch.autograd import grad # import grad function from autograd, allowing the computation
# of gradient of a scalar wrt some tensors.
grad_L_w1 = grad(loss, w1, retain_graph = True) # calculates the gradient of
# loss (scalar o/p from logistic regression forward pass) wrt 'w1' tensor.
# retain_graph = True preserves the computational graph for future gradient computation.
# In simple terms, it is the rate of change of w1 wrt loss.
grad_L_b = grad(loss, b, retain_graph = True) # calculate the gradient of loss wrt 'b' (bias)
# tensor. retain_graph = True preserves the computational graph for future gradient computation.
print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [12]:
loss.backward() # .backward() performs back propogation starting from the specified tensor, here,
# loss. It traverses the computational graph & automatically calculates the gradients of loss
# with respect to all tensors. Here, it computes d(loss)/d(w1) and d(loss)/d(b)
print(w1.grad)
print(b.grad)

tensor([-0.0898])
tensor([-0.0817])


In [13]:
# MULTILAYER PERCEPTRON
class NeuralNetwork(torch.nn.Module): # custom neural network class that inherits from PyTorch's base Module Class
  def __init__(self, num_inputs, num_outputs): # constructor method where the neural network architeecture is defined
    super().__init__() # calls the constructor of the parent class (torch.nn.Module)

    self.layers = torch.nn.Sequential( # define a Sequence of layers and operations
        # HIDDEN LAYER 1
        torch.nn.Linear(num_inputs, 30),
# first fully connected dense layer.
# Takes the input of num_inputs features and transforms them into 30 output features.
        torch.nn.ReLU(),
# ReLu activation function that intrduces non linearity into the model.
# Without a non linear activation function, neural networks will only learn linear relationships.
# ReLu outputs the input if its positive, else zero i.e., f(x) = max (0, x)

        # HIDDEN LAYER 2
        torch.nn.Linear(30, 20),
# 30 input features are transformed into 20 output features in the second hidden layer.
        torch.nn.ReLU(),

        # OUTPUT LAYER
        torch.nn.Linear(20, num_outputs),
# output layer; 20 features from the previous (last) hidden layer and transforms them into num_outputs features.
# num_outputs would typically correspond to the number of classes in a classification problem.
    )
  def forward(self, x):
  # defines the forward pass of the network. Describes how input data 'x' flows
  # through the layers to produce an output.
    logits = self.layers(x)
  # When NeuralNetwork object (ex: model(input_data)) is called, this forward method
  # is implicitly executed. The input tensor 'x' is passed sequentially through all the layers defined in self.layers()
    return logits

In [14]:
model = NeuralNetwork(50,3)

In [15]:
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [16]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
# calculates the total number of trainable parameters within the model
# p.numel() - method returns the total number of elements (individual scalar values)
# in that parameter tensor. For example, if a weight matrix is 50x30, numel() would return 1500.
print(num_params)

2213


In [17]:
torch.manual_seed(42)
# manual seed is crucial for reproducibility; When you initialize neural network layers
# (like torch.nn.Linear), their weights and biases are typically initialized randomly.
# By setting a seed, you ensure that every time you run this code, the random
# initialization will be the same, leading to identical starting weights and biases.
# This makes your experiments consistent and allows others to reproduce your results.
model = NeuralNetwork(50,3)
print(model.layers[0].weight)
print(model.layers[0].bias)

Parameter containing:
tensor([[ 0.1081,  0.1174, -0.0331,  ...,  0.0253,  0.0718, -0.0862],
        [-0.1400, -0.0546, -0.1085,  ..., -0.0477, -0.0501, -0.1368],
        [-0.0810,  0.0353, -0.0187,  ...,  0.1142,  0.1288, -0.1121],
        ...,
        [-0.0031, -0.0573,  0.0515,  ...,  0.0271, -0.0928, -0.1175],
        [-0.0444, -0.1318, -0.0660,  ...,  0.0647, -0.1230, -0.0531],
        [ 0.0023, -0.1223,  0.0797,  ...,  0.0369,  0.0862,  0.1328]],
       requires_grad=True)
Parameter containing:
tensor([ 0.1205, -0.0081,  0.0274, -0.0846,  0.0547, -0.1356,  0.0667,  0.0259,
         0.0435,  0.0518, -0.0649,  0.1214,  0.0675,  0.0929,  0.0187,  0.0888,
         0.0952,  0.0641,  0.1077, -0.0257,  0.0930,  0.0326,  0.0851,  0.1377,
         0.0320, -0.0525, -0.0141,  0.0398,  0.0238, -0.0386],
       requires_grad=True)


How torch.manual_seed() Ensures Sameness:
- Pseudo-random Numbers:
  - Computers don't generate truly random numbers. Instead, they use algorithms to generate sequences of numbers that appear random.
  - These are called pseudo-random numbers.
- The Seed:
  - A pseudo-random number generator starts with an initial value called a 'seed'.
  - If you provide the same seed to the generator, it will produce the exact same sequence of 'random' numbers every single time.
- Reproducible Initialization:
  - When you call torch.manual_seed(42), you're telling PyTorch's random number generator to start its sequence with '42'.
  - Then, when you create a new NeuralNetwork instance (e.g., model = NeuralNetwork(50,3)), the weights and biases inside its torch.nn.Linear layers are initialized using the next numbers in that fixed pseudo-random sequence.
  - Because the sequence is always the same for a given seed, the initial weights and biases of your model will be identical every time you run that code, as long as the seed is set before the model is instantiated.

In [18]:
# code block demonstrates how to generate a random input, pass it through your
# previously defined NeuralNetwork model, and print the resulting output.

torch.manual_seed(123)
# crucial for ensuring that the random input tensor X generated in the next step is reproducible.

X = torch.rand((1,50))
# torch.rand() function generates a tensor with random numbers uniformly sampled from the interval [0, 1).
#  The (1, 50) argument specifies the shape of the tensor:
# it creates a tensor with 1 row and 50 columns. This shape is important because your
# NeuralNetwork was initialized with num_inputs=50, meaning it expects an input
# tensor with 50 features per sample. The '1' in (1,50) typically represents a batch size
# of 1, meaning we are processing a single input sample.

out = model(X)
# This line performs the forward pass through your NeuralNetwork model.
# When you call the model instance like a function (e.g., model(X)),
# PyTorch automatically executes the forward method that you defined within your NeuralNetwork class.
# The input tensor X goes through all the layers (Linear, ReLU, Linear, ReLU, Linear) in the
# self.layers sequential container, and the final output is stored in the out variable.
# The out tensor contains the 'logits' from the network's final layer.


print(out)
# the output will be a tensor of shape (1, 3)
# (one sample, three outputs), as your NeuralNetwork was initialized with num_outputs=3.

tensor([[ 0.1019, -0.0396, -0.1432]], grad_fn=<AddmmBackward0>)


In [19]:
# Creating a small toy dataset
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])

y_train = torch.tensor([0,0,0,1,1])

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6]
])

y_test = torch.tensor([0,1])

In [21]:
# Defining a custom Dataset class
from torch.utils.data import Dataset

class ToyDataset(Dataset):
  def __init__(self, X, y): # setup attributes awe can access later in __getitem__ and __len__ methods
    self.features = X
    self.labels = y

  def __getitem__(self, index): # define instructions for returning exactly one item from the dataset via an index.
  #this refers to the features and the class label corresponding to a single training example or test instance.
    one_x = self.features[index]
    one_y = self.labels[index]
    return one_x, one_y

  def __len__(self): # contains instructions for retrieving the length of the dataset.
    return self.labels.shape[0] # .shape used to return the number of rows in the deature array.

In [22]:
train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [23]:
# This code block uses the built-in len() function to determine the number of samples in your train_ds and test_ds datasets.
print(len(train_ds))
print(len(test_ds))

5
2


In [24]:
# Instatntiating data loaders

from torch.utils.data import DataLoader
torch.manual_seed(123)

train_loader = DataLoader(
    dataset = train_ds,
    batch_size = 2, # data should be grouped into batches of 2 samples
    shuffle = True, # reshuffle the order of the samples in train_ds at the beginning of each epoch
    num_workers = 0 # how many subprocesses to use for data loading. i.e., background processes
)

test_loader = DataLoader(
    dataset = test_ds,
    batch_size = 2,
    shuffle = False,
    num_workers = 0
)

In [25]:
for idx, (x,y) in enumerate(train_loader):
  print(f"Batch {idx + 1}: ", x, y)

Batch 1:  tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2:  tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 3:  tensor([[ 2.7000, -1.5000]]) tensor([1])
